# evaluate the fine tuned model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import AutoPeftModelForCausalLM
import torch
from datasets import load_dataset, Dataset
import multiprocessing
import json
import time

import transformers
transformers.logging.set_verbosity_info()

In [ ]:
# load the base model
checkpoint = "HuggingFaceTB/SmolLM-360M"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
print(tokenizer.pad_token, tokenizer.eos_token, tokenizer.unk_token)
print(tokenizer.pad_token_id, tokenizer.eos_token_id, tokenizer.unk_token_id)

In [ ]:
# config for smollm 360m
# set the pad token as eos token so that the finetuned model knows when to stop generating tokens
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
print(tokenizer.pad_token, tokenizer.eos_token, tokenizer.unk_token)
print(tokenizer.pad_token_id, tokenizer.eos_token_id, tokenizer.unk_token_id)

In [ ]:
model_path = "lora_weights/checkpoint-500"
peft_model = AutoPeftModelForCausalLM.from_pretrained(model_path, device_map="cpu", torch_dtype=torch.float16)

# * merge the adapter onto the base model itself
merged_model = peft_model.merge_and_unload()
# The adapters are merged now and it is transformers class again
print(type(merged_model))

print(merged_model.num_parameters() / 1e6, " M Params")

In [ ]:
# read the valid dataset
valid_data = []
with open('data/valid.jsonl', 'r') as file:
    for line in file:
        valid_data.append(json.loads(line))


In [ ]:
valid_data[1]["text"]

# single inference

In [ ]:
ind = 15

input_text = f"<user>{valid_data[ind]['event_text']}</user><output>"

inputs = tokenizer(input_text, return_tensors="pt")
print(f"\nInput Text : {input_text}")
print(f"Length of input tokens processed : {len(inputs['input_ids'][0])}")

output_tokens = merged_model.generate(**inputs, max_new_tokens=100, do_sample=True, temperature=0.1, pad_token_id=tokenizer.eos_token_id)

print(f"Length of output tokens generated : {len(output_tokens[0]) - len(inputs['input_ids'][0])}")

truncated_output = tokenizer.decode(output_tokens[0][len(inputs[0]):])
output = tokenizer.decode(output_tokens[0])

print(f"Output text : {truncated_output}")
resp = output.split("</output>")[0].split("<output>")[1]

pred = json.loads(resp)

In [ ]:
print(pred)